In [1]:
import torch
import torch.nn as nn
import soundfile
import librosa
import torchlibrosa
from panns_inference import AudioTagging
#STFT - Short-time Fourier transform

#The short-time Fourier transform (STFT) is a Fourier-related transform used to determine the sinusoidal 
#frequency and phase content of local sections of a signal as it changes over time.[1] In practice, the 
#procedure for computing STFTs is to divide a longer time signal into shorter segments of equal length and 
#then compute the Fourier transform separately on each shorter segment. This reveals the Fourier spectrum 
#on each shorter segment. One then usually plots the changing spectra as a function of time, 
#known as a spectrogram or waterfall plot, such as commonly used in software defined radio (SDR) based 
#spectrum displays. Full bandwidth displays covering the whole range of an SDR commonly use fast Fourier transforms (FFTs).

In [2]:
model = torch.load("artifacts/Cnn14_mAP=0.431.pth", map_location="cpu")

In [19]:
import librosa
import numpy as np
from panns_inference import AudioTagging, labels

audio_path = r"C:/Users/RomanKyrkalo/Downloads/download.wav"

# 1) Load mono audio at 32 kHz
y, sr = librosa.load(audio_path, sr=32000, mono=True)

# 2) Add batch dimension: (1, T)
y = y[None, :]

# 3) Run inference
model = AudioTagging(checkpoint_path=None)  # will download CNN14 weights on first use
clipwise_scores, embedding = model.inference(y)  # clipwise_scores: (1, 527)

# 4) Show top-10 labels
topk = np.argsort(-clipwise_scores[0])[:10]
for k in topk:
    print(labels[k], ": ", float(clipwise_scores[0, k]))


Checkpoint path: C:\Users\RomanKyrkalo/panns_data/Cnn14_mAP=0.431.pth
GPU number: 1
Bird vocalization, bird call, bird song :  0.5647661089897156
Environmental noise :  0.41335853934288025
Bird :  0.38755905628204346
Chirp, tweet :  0.3797639310359955
Outside, rural or natural :  0.0777573436498642
Music :  0.06243884563446045
Animal :  0.030938906595110893
Speech :  0.027243217453360558
Owl :  0.020749123767018318
Insect :  0.016712162643671036
